In [ ]:
# https://platform.openai.com/docs/guides/function-calling?api-mode=responses#defining-functions
# https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [59]:
import os 
import json
from dotenv import load_dotenv
from openai import AsyncOpenAI, OpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client, 
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

In [60]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

client2 = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

In [61]:
def message(role, content):
    return {"role": role, "content":
            [{
                "type": "text",
                "text": content}]
    }




<img src="https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png" alt="Function Calling" width="400px"/>

In [62]:
messages = [
    message("developer", "you are a helpful assistant"),
    message("user", "what is the weather in Sydney?"),
]

In [63]:

completion = await client.chat.completions.create(
    model="gpt-4o",
    messages=messages)



print(completion.choices[0].message.to_json())

{
  "content": "I don’t have real-time weather data, but you can easily check the current weather in Sydney by using any weather service or app, such as:\n\n- [Weather.com](https://www.weather.com/)\n- [AccuWeather](https://www.accuweather.com/)\n- [Bureau of Meteorology (Australia)](http://www.bom.gov.au/)\n\nYou can also ask your phone's virtual assistant, like Siri, Google Assistant, or Alexa, for the most up-to-date weather information. Let me know if you'd like help with anything else!",
  "refusal": null,
  "role": "assistant"
}


In [64]:
get_weather_function_definition = {
    "type": "function",
    "function":{
        "name": "get_weather",
        "description": "Get the current weather in a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the location to get the weather for",
                }
            },
            "required": ["location"],
            "additionalProperties": False,

        },
    },
    "strict": True
}  

tools = [get_weather_function_definition]

In [65]:
def get_weather(location):
    # return f"The weather in {location} is sunny with a high of 25°C."
    return {"temperature": 25, "condition": "sunny"}

In [66]:
chat_history = messages
completion = await client.chat.completions.create(
    model="gpt-4o", 
    messages=chat_history,
    tools=tools
)

In [67]:
print(completion.to_json())

{
  "id": "chatcmpl-BGOdSAgft2XuJOgYOvV8dE07sTDZN",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_ctvGVkIb0ufFGMFBqCMCGxjn",
            "function": {
              "arguments": "{\"location\":\"Sydney\"}",
              "name": "get_weather"
            },
            "type": "function"
          }
        ]
      },
      "content_filter_results": {}
    }
  ],
  "created": 1743247646,
  "model": "gpt-4o-2024-11-20",
  "object": "chat.completion",
  "system_fingerprint": "fp_ded0d14823",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 71,
    "total_tokens": 86,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": 

In [68]:
chat_history.append(json.loads(completion.choices[0].message.to_json()))  # append model's function call message
chat_history

[{'role': 'developer',
  'content': [{'type': 'text', 'text': 'you are a helpful assistant'}]},
 {'role': 'user',
  'content': [{'type': 'text', 'text': 'what is the weather in Sydney?'}]},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
    'function': {'arguments': '{"location":"Sydney"}', 'name': 'get_weather'},
    'type': 'function'}]}]

In [69]:
finish_reason = completion.choices[0].finish_reason
print(finish_reason)
tool_calls = completion.choices[0].message.tool_calls
print(tool_calls[0].to_json())



tool_calls
{
  "id": "call_ctvGVkIb0ufFGMFBqCMCGxjn",
  "function": {
    "arguments": "{\"location\":\"Sydney\"}",
    "name": "get_weather"
  },
  "type": "function"
}


In [77]:

for tool in tool_calls:
    print(tool.id)
    print(tool.function.name)
    print(tool.function.arguments)
    function_name = tool.function.name
    function_args = json.loads(tool.function.arguments)
    tool_call_id = tool.id



    # execute the function call and append the results as a ToolMessage in the message history
    function_call_results = locals()[function_name](**function_args)
    print(function_call_results)
    function_call_results_message = {
        "role": "tool",
        "type": "function_call_output",
        "tool_call_id": tool_call_id,
        "content": str(function_call_results)
    }

        #    messages.append({
        #     "role":"tool", 
        #     "tool_call_id":tool_call_id, 
        #     "name": tool_function_name, 
        #     "content":results
        # })

    chat_history.append(function_call_results_message)

call_ctvGVkIb0ufFGMFBqCMCGxjn
get_weather
{"location":"Sydney"}
{'temperature': 25, 'condition': 'sunny'}


In [78]:
str(function_call_results)

"{'temperature': 25, 'condition': 'sunny'}"

In [79]:
chat_history

[{'role': 'developer',
  'content': [{'type': 'text', 'text': 'you are a helpful assistant'}]},
 {'role': 'user',
  'content': [{'type': 'text', 'text': 'what is the weather in Sydney?'}]},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
    'function': {'arguments': '{"location":"Sydney"}', 'name': 'get_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'type': 'function_call_output',
  'tool_call_id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
  'content': "{'temperature': 25, 'condition': 'sunny'}"}]

In [74]:
chat_history.pop(-1)

{'role': 'tool',
 'type': 'function_call_output',
 'tool_call_id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
 'content': {'temperature': 25, 'condition': 'sunny'}}

In [81]:
response_2 = await client.chat.completions.create(
    model="gpt-4o",
    messages=chat_history,
    tools=tools
)


print(response_2.choices[0].message.to_json())

chat_history.append(json.loads(response_2.choices[0].message.to_json()))  # append model's function call message

{
  "content": "The current weather in Sydney is 25°C and sunny.",
  "refusal": null,
  "role": "assistant"
}


In [83]:
chat_history.append(message("user", "what again in Arabic please?"))

In [84]:
chat_history

[{'role': 'developer',
  'content': [{'type': 'text', 'text': 'you are a helpful assistant'}]},
 {'role': 'user',
  'content': [{'type': 'text', 'text': 'what is the weather in Sydney?'}]},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
    'function': {'arguments': '{"location":"Sydney"}', 'name': 'get_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'type': 'function_call_output',
  'tool_call_id': 'call_ctvGVkIb0ufFGMFBqCMCGxjn',
  'content': "{'temperature': 25, 'condition': 'sunny'}"},
 {'content': 'The current weather in Sydney is 25°C and sunny.',
  'refusal': None,
  'role': 'assistant'},
 {'role': 'user',
  'content': [{'type': 'text', 'text': 'what again in Arabic please?'}]}]

In [85]:
response_3 = await client.chat.completions.create(
    model="gpt-4o",
    messages=chat_history,
    tools=tools
)


print(response_3.choices[0].message.to_json())

{
  "content": "الطقس الحالي في سيدني هو 25 درجة مئوية ومشمس.",
  "refusal": null,
  "role": "assistant"
}
